# Imports

In [23]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_regression
import xgboost
from sklearn.datasets import make_regression

# Linear regression

In [22]:
# Generate a sample dataset
X, y = make_regression(n_samples=100, n_features=2, noise=10, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Perform k-fold cross-validation (let's say k=5)
cross_val_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

new_lin_scores = - cross_val_scores

# Display the cross-validation scores
print("Cross-Validation Scores:")

print(new_lin_scores)

# Calculate and print the average performance metric
average_score = np.mean(new_lin_scores)
print(f"\nAverage Performance Metric: {average_score:.2f}")


Cross-Validation Scores:
[ 78.03947316 177.14216897 127.50389178 111.63476136  97.84311545]

Average Performance Metric: 118.43


# The Ensemble Models :  The XGBRegressor

In [3]:
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, random_state=42)
base_model = xgboost.XGBRegressor(objective='reg:squarederror', random_state=42)


In [29]:
hyper_parameter_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}
#  'n_estimators': 200, sets the number of boosting rounds for this model (the number of decision trees to be used here). the performance of the model in the first decision tree is used to boost the model in the second decision tree  and s on .
# To note again , the cv for this case specifies the number of epochs the model will go though the whole dataset,  and then it aggregates the results to find the best parameters.


grid_search =GridSearchCV(estimator=base_model, param_grid= hyper_parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=42, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 0.9, 1.0],
                                        'learning_rate': [0.01, 0.1, 0.2],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [50, 100, 200],
                                        'subsample': [0.8, 0.9, 1.0]},
                   scoring='neg_mean_squared_error')

In [30]:
best_params = grid_search.best_params_
best_params

{'subsample': 1.0,
 'n_estimators': 100,
 'min_child_weight': 3,
 'max_depth': 3,
 'learning_rate': 0.1,
 'colsample_bytree': 1.0}

In [33]:
static_best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.2,
    'max_depth': 3,
    'min_child_weight': 5,
    'n_estimators': 200,
    'subsample': 0.8
}
final_model = xgboost.XGBRegressor(objective='reg:squarederror', random_state=42, **static_best_params)
final_model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [34]:
import pandas as pd

results = final_model.predict(X)
resultDf = pd.DataFrame(data = {
    'Actual':y,
    'Predictions':results.flatten(),
    
})
resultDf

,Actual,Predictions
0,-49.265023,-49.701683
1,-85.162387,-82.630119
2,211.225658,212.842987
3,29.196562,30.214071
4,84.348096,85.980270
...,...,...
95,147.505343,148.590652
96,32.710111,33.245121
97,-141.334380,-138.956512
98,48.782598,48.125698
